In [2]:
import numpy as np
import math
import cmath
import matplotlib.pyplot as plt
pi = math.pi

#Prints out numbers without "np.flat64" displaying
np.set_printoptions(legacy='1.25')

## Problem 8.16

In [15]:
# Constants
epsilon_0 = 8.854*10**-12 #permitivity of free space
mu_0 = 4*pi*10**-7 #permeability of free space
c = 2.998*10**8 #speed of light

# Generic Equations

def freq_to_omega(freq) : #get angular frequency (rad/s)
    return 2*pi*freq

def Np_to_dB(Np) : # converts Nepers to Decibels.
    return Np*20/(np.log(10))

#mu_r, epsilon_r = relative properties of medium
#freq = frequency of propagation
def calc_wavenumber(mu_r, epsilon_r, freq) : # wavenumber, k
    return freq_to_omega(freq)*np.sqrt(mu_r*mu_0*epsilon_r*epsilon_0)

# k = wavenumber
def calc_SL_guide_wavelength(k) : # lambda_g
    return 2*pi/k

#frequency = frequency of propagation
#conductance = property of conductor, sigma
def calc_surface_resistance(frequency,conductance) :
    return np.sqrt((freq_to_omega(frequency)*mu_0)/(2*conductance))


#Microstrip Equations

#epsilon_r = relative permitivity of dialectric
#Zo = desired characteristic impedence of line
#d = substrate thickness
def calc_MS_width(epsilon_r, Zo,d) :
    A = (Zo/60)*(np.sqrt((epsilon_r+1)/2)) + ((epsilon_r-1)/(epsilon_r+1))*(0.23 + (0.11)/epsilon_r)
    B = 377*pi/(2*Zo*np.sqrt(epsilon_r))
    # print(f'A = {A}')
    # print(f'B = {B}')
    W = (8*np.exp(A)/(np.exp(2*A)-2))*d
    if ((W/d) <= 2) & (W>0) :
        #print(f'W/d <= 2')
        return W
    else :
        W = ((2/pi)*(B - 1 - np.log(2*B-1) + ((epsilon_r-1)/(2*epsilon_r))*(np.log(B-1) + 0.39 - (0.61/epsilon_r)) ))*d
        if (W/d >= 2) : return W
        else :
            print(f'calc_MS_width ERROR: Zo = {Zo} and epsilon_r = {epsilon_r} invalid. W/d = {W/d}')
    
#epsilon_r = relative permitivity of dialectric
#W = width of copper microstrip (not groundplane)
#d = substrate thickness
def calc_MS_epsilon_effective(epsilon_r, W, d) :
    return ((epsilon_r+1)/(2) + ((epsilon_r-1)/(2))*(1/(np.sqrt(1 + 12*(d/W)))))

#epsilon_r = relative permitivity of dialectric
#tandelta = property of dialectric
#W = width of copper microstrip (not groundplane)
#d = substrate thickness
def calc_MS_dialectric_loss(epsilon_r,freq, tandelta, W, d) :
    k0 = (2*pi*freq/c)
    epsilon_e = calc_MS_epsilon_effective(epsilon_r, W, d)
    return ((k0*epsilon_r*(epsilon_e-1)*tandelta)/(2*np.sqrt(epsilon_e)*(epsilon_r-1)))

#Rs = surface resistance of conductor
#Zo = characteristic impedence of line.
#W = width of copper microstrip (not groundplane)
def calc_MS_conductor_loss(Rs, Zo, W) : 
    return (Rs/(Zo*W))

#Zo = characteristic impedence of line.
#epsilon_r = relative permitivity of dialectric
#freq = frequency of line.
#d = substrate thickness
#tandelta = property of dialectric
#conductance = conductance of conductor.
def calc_MS_total_loss(Zo, epsilon_r, freq, d, tandelta, conductance) :
    Rs = calc_surface_resistance(freq,conductance)
    W = calc_MS_width(epsilon_r, Zo, d)
    #print(f'W = {W}')
    cond_loss = Np_to_dB(calc_MS_conductor_loss(Rs, Zo, W))
    dialectric_loss = Np_to_dB(calc_MS_dialectric_loss(epsilon_r, freq, tandelta, W, d))
    #print(f'Conductor Loss = {cond_loss} dB/m')
    #print(f'Dialectric Loss = {dialectric_loss} dB/m')
    return cond_loss + dialectric_loss

In [17]:
Zl = 10
Zh = 120
freq = 3*10**9

eleclength_C1 = 1.7058*Zl/50
eleclength_L2 = 1.2296*50/Zh
eleclength_C3 = 2.5408*Zl/50
eleclength_L4 = eleclength_L2
eleclength_C5 = eleclength_C1

electrical_lengths = [eleclength_C1,eleclength_L2,eleclength_C3,eleclength_L4,eleclength_C5]

for x in electrical_lengths:
    print(180*x/pi)

19.547028138683167
29.35453770386918
29.115423317367917
29.35453770386918
19.547028138683167


In [16]:
epsilon_r = 2
d = 0.00079
tandelta = 0.02
freq_o = 3*10**9 #Hz

ms_widths = [0] * len(electrical_lengths) #
ms_lengths = [0] * len(electrical_lengths) #length of each segment

index = 0
#stepped impedence width and length calculations
for width in range(0,len(ms_widths)) :
    if (index % 2) == 0 : Z = 10 #low impedence
    else : Z = 120 #high impedence
    W = calc_MS_width(epsilon_r, Z,d)
    ms_widths[index] = W
    epsilon_effective = calc_MS_epsilon_effective(epsilon_r, W, d)
    length = (electrical_lengths[index]*c)/(freq_o*2*pi*np.sqrt(epsilon_effective))
    ms_lengths[index] = length
    index+=1

print(ms_widths)
print(ms_lengths)

[0.01881321015590829, 0.0005023966700655357, 0.01881321015590829, 0.0005023966700655357, 0.01881321015590829]
[0.005892813987304051, 0.006417675098070259, 0.00877738408895658, 0.006417675098070259, 0.005892813987304051]
